In [1]:
from collections import defaultdict
topic2article = defaultdict(list)

#  Create a dictionary of the form: {topic1: [article number, ....], topic2: [article number, ....]}
# So each topic is associated with one or more article, I do it only for the first 2 topics.

def group_articles(articles_info):
    with open (articles_info) as file_in:
        next(file_in)
        for line in file_in.readlines()[:28]:
            article_info = line.split("\t")
            topic2article[article_info[0]].append(article_info[2].strip())
    return  topic2article

    

In [2]:
topic2article = group_articles('IBM_Debater_(R)_CE-EMNLP-2015.v3/articles.txt')
print(topic2article)

defaultdict(<class 'list'>, {'This house believes that the sale of violent video games to minors should be banned': ['567', '524', '542', '1', '503', '508', '507', '520', '506', '525', '522', '504', '568', '570', '505'], 'This house supports the one-child policy of the republic of China': ['533', '498', '485', '514', '518', '495', '461', '21', '1282', '517', '532', '531', '484']})


In [3]:

import os
from pathlib import Path
import re

from nltk import sent_tokenize

# Save content of aricles of the form {article_number: list of sentences of the article}
def read_articles(ARTICLES_DIR):
    article_content = {}
    for file_name in os.listdir(ARTICLES_DIR):
        txt_path = Path(ARTICLES_DIR, file_name)
        
#         Problem DS.Store
        if str(txt_path)[-3:] == "txt":
        
            number = re.findall('\d+', str(txt_path))[-1]
            with open (txt_path, encoding="'utf-8'") as file_in:
                content = file_in.read()
                sents = sent_tokenize(content)
                article_content[number] = sents

    return article_content

    

#     WHOLE DATASET
# ARTICLES_DIR = '/Users/olhasvezhentseva/Documents/IBM_Debater_(R)_CE-EMNLP-2015.v3/articles'

# I read only a set of articles to save time, I put articles #1, 485, 498 + from 500 to 600 in the folder 'sample_articles', 
# but it doesn't matter now. I chose these articles because they contain articles related to the first 2 topics 
# (video games and China policy on chidren).

# SAMPLE DATASET
ARTICLES_DIR = 'sample_articles'



In [4]:

articles_content = read_articles(ARTICLES_DIR)
articles_content



{'543': ['.',
  'The inclusion of sex and nudity in video games has been a controversial topic since the early days of the industry.',
  'While many games have used scantily clad images or characters to sell or enhance games (see sex in advertising), this article deals with games that use sex acts or nudity as a character motivation, in-game reward, or simply as a gameplay element.',
  'These games originate worldwide, on most platforms and in most genres.',
  'While releases in Europe and North America have been sporadic, Japan has seen the emergence of a distinct genre—the eroge (or erotic game).',
  'One of the earliest video games (if not the first) to feature sexual themes was the 1981 text-based Softporn Adventure, published by On-Line Systems for the Apple II.',
  "Despite heavy piracy the game still sold 25,000 copies, roughly equivalent to 25% of the number of Apple II's sold at the time.",
  'In a 1981 article in Time Magazine, On-Line reported that they were making a version

In [5]:
# Create dict of the form: {"topic":  {claim1: [], claim2: [], ...}}
# So that we see every topic and claims associated with it
# The empty list will be filled later with corresponding evidences
def topic2claim(claim_file):
    topic2claim = defaultdict(lambda: dict())

    with open (claim_file) as file_in:
        next(file_in)
        for line in file_in.readlines()[:140]:
            claims_info = line.split("\t")
            topic = claims_info[0]
            claim = claims_info[1].strip()
            topic2claim[topic][claim] = []
    return topic2claim



    


In [6]:
topic2claim = topic2claim('IBM_Debater_(R)_CE-EMNLP-2015.v3/claims.txt')

In [75]:
topic2claim

defaultdict(<function __main__.topic2claim.<locals>.<lambda>()>,
            {'This house believes that the sale of violent video games to minors should be banned': {'Exposure to violent video games causes at least a temporary increase in aggression and this exposure correlates with aggression in the real world': [],
              'video game violence is not related to serious aggressive behavior in real life': [],
              'some violent video games may actually have a prosocial effect in some contexts': [],
              'exposure to violent video games causes both short term and long term aggression in players and decreases empathy and prosocial behavior': [],
              'Violent video games increase the violent tendencies among youth': [],
              'No conclusive link was found between video game usage and violent activity': [],
              'violent video games are significantly associated with: increased aggressive behavior, thoughts, and affect; increased physiologi

In [7]:

# Fill in claims it topic2claim dict, so that it is of the form {"topic":  {claim1: [EVIDENCE1, EVIDENCE2]}   
def connect_claim_evidence(evidence_file, topic2claim ):

    with open (evidence_file) as file_in:
            for line in file_in.readlines()[:545]:
                evidence_info = line.split("\t")
                topic = evidence_info[0]
                claim = evidence_info[1].strip()
                evidence = evidence_info[2]

                topic2claim[topic][claim].append(evidence)
                
    return topic2claim

In [8]:
claim2evidence = connect_claim_evidence('IBM_Debater_(R)_CE-EMNLP-2015.v3/evidence.txt', 
                                      topic2claim )

In [78]:
claim2evidence

defaultdict(<function __main__.topic2claim.<locals>.<lambda>()>,
            {'This house believes that the sale of violent video games to minors should be banned': {'Exposure to violent video games causes at least a temporary increase in aggression and this exposure correlates with aggression in the real world': ['A 2001 study found that exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world',
               'The most recent large scale meta-anlysis-- examining 130 studies with over 130,000 subjects worldwide-- concluded that exposure to violent video games causes both short term and long term aggression in players',
               'One study did find an increase in reports of bullying, noting, "Our research found that certain patterns of video game play were much more likely to be associated with these types of behavioral problems',
               'when one combines all relevant empirical 

So now we have 3 variables:

topic2article -- topics with associciated articles' numbers

articles_content -- article's number with its content

claim2evidence -- topics with corresponding claims, each claim is supported with 0 or more evidences